In [1]:
import sys
import os
from timeit import default_timer as timer
os.getcwd()

'/home/jupyter/Notebooks/crystal/NLP/nlp_testing'

In [ ]:
# Adapted from the tutorial at https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

In [2]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
# list(tokenizer.vocab.keys())[5000:5020]
list(tokenizer.vocab.keys())[17733]

'disgusted'

In [ ]:
# Define a new example sentence with multiple meanings of the word "disgusted"
text = "[CLS] She made a disgusted pout [SEP] Her disgusted expression was contagious [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [ ]:
# Recreate vocabulary words from their tokenized representations.
for t in tokenized_text:
    this_word = ''
    for token in t:
        this_word += token.strip('#')
#     print(this_word)

In [ ]:
# Mark each of the tokens as belonging to sentence "0" or "1".

segments_ids = [1] * len(tokenized_text[3])
# segments_ids = [0,0,0]
print (segments_ids)
print(indexed_tokens)
print(tokenized_text[3])

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens[3]])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 1

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

In [ ]:
# For our token, select its feature values from layer 5.
token_i = 1
layer_i = 5
vec = encoded_layers[layer_i][batch_i][token_i]
# print(vec)

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()

In [ ]:
# `encoded_layers` is a Python list.
print('     Type of encoded_layers: ', type(encoded_layers))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', encoded_layers[0].size())

In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings.size()

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

In [ ]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

In [23]:
# Concatenate the last 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_last = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_last.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_last), len(token_vecs_cat_last[0])))
print(token_vecs_cat_last[0])

Shape is: 1 x 3072
tensor([-0.2373,  0.8259, -0.6190,  ..., -0.3836, -0.5039,  0.6153])


In [24]:
# Sum the last 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_last = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_last.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_last), len(token_vecs_sum_last[0])))

Shape is: 1 x 768


In [ ]:
# Concatenate the first 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_first = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[0], token[1], token[2], token[3]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_first.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_first), len(token_vecs_cat_first[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the first 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_first = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[:4], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_first.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_first), len(token_vecs_sum_first[0])))

In [ ]:
# Concatenate the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_middle1 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[4], token[5], token[6], token[7]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_middle1.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_middle1), len(token_vecs_cat_middle1[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_middle1 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[4:8], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_middle1.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_middle1), len(token_vecs_sum_middle1[0])))

In [ ]:
# Concatenate the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_middle2 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[8], token[9], token[10], token[11]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_middle2.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_middle2), len(token_vecs_cat_middle2[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_middle2 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[8:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_middle2.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_middle2), len(token_vecs_sum_middle2[0])))

In [ ]:
# Concatenate all hidden layers to create word embeddings.
token_vecs_cat_all = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[0], token[1], token[2], token[3], token[4], token[5], token[6], token[7], token[8], token[9], token[10], token[11]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_all.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_all), len(token_vecs_cat_all[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum all hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_all = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_all.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_all), len(token_vecs_sum_all[0])))

In [ ]:
# Make a single vector to represent the pair of sentences by averaging across tokens.
# `encoded_layers` has shape [12 x 1 x 22 x 768]
sentences_vec = []
# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = encoded_layers[11][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
for s in sentence_embedding:
    sentences_vec.append(s)
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())
print(sentence_embedding[767])
print(sentence_embedding[-1])
print(f'Shape of sentences vector is: {len(sentences_vec)}')
print(sentences_vec[-1])

In [ ]:
from scipy.spatial.distance import cosine
# Test the similarity of a word with itself.
# For words trained contextually, self-synonymy is less than 1.
similarity = 1 - cosine(token_vecs_cat[0], token_vecs_cat[0])
print(f'Similarity of {tokenized_text[8]} and {tokenized_text[8]} in token_vecs_cat is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum[4], token_vecs_sum[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_first[4], token_vecs_cat_first[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_first is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_first[4], token_vecs_sum_first[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_first is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_middle1[4], token_vecs_cat_middle1[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_middle1 is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_middle1[4], token_vecs_sum_middle1[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_middle1 is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_middle2[4], token_vecs_cat_middle2[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_middle2 is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_middle2[4], token_vecs_sum_middle2[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_middle2 is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_all[4], token_vecs_cat_all[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_all is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_all[4], token_vecs_sum_all[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_all is: {similarity}')

In [10]:
######################################################################################
############## BEGIN TESTING STATIC CONTEXTUAL EMBEDDING CREATION ####################
######################################################################################

In [11]:
def make_vocab(vocab_file):
    # start = timer()
    vocab = []
    # vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/vocab_files/vocab_checked.txt'
    with open(vocab_file, 'r') as v:
        vocab = v.read().splitlines()
    # end = timer()
    # run_time = end - start
#     print(f'There are {len(vocab)} words in the vocabulary.\n')
#     print(f'It took {run_time} seconds to read the vocabulary file into memory.')
#     print(f'Test word is {vocab[2]}.')
    return vocab

In [12]:
def tokenize_text(vocab):
    tokenized_text = []
    indexed_tokens = []
    for word in vocab:
        # Add the special tokens.
    #     marked_text = "[CLS] " + word + " [SEP]"
        marked_text = word

        # Split the sentence into tokens.
        # tokenized_text = tokenizer.tokenize(marked_text)
        tokenized_text.append(tokenizer.tokenize(marked_text))
#         print(f'Added {tokenized_text[-1]} to the tokenized_text array.')


        # Map the token strings to their vocabulary indeces.
        indexed_tokens.append(tokenizer.convert_tokens_to_ids(tokenized_text[-1]))

        # Display the words with their indeces.
    #     print(f'The word {tokenized_text[-1][1]} is at index {indexed_tokens[-1]}.')
#         for tup in zip(tokenized_text[-1], indexed_tokens[-1]):
#             print('{:<12} {:>6,}'.format(tup[0], tup[1]))
    return tokenized_text, indexed_tokens

In [13]:
def generate_segments_IDs(tokenized_text):
    # Create segment IDs for sentence 1 (there can be a sentence 0 to compare to
    # sentence 1, but we're not doing that).
    # Check that indices and token indices look correct.
    segments_IDs = []
    for i in range(len(tokenized_text)):
        segments_IDs.append([1] * len(tokenized_text[i]))
#     for i in range(len(segments_IDs)):
#         print (segments_IDs[i])
#         print(tokenized_text[i])
    return segments_IDs

In [14]:
def generate_embeddings(indexed_tokens, segments_IDs):
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_IDs])

    # Load pre-trained model (weights)
    model = BertModel.from_pretrained('bert-base-uncased')

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
#         print('Type of encoded_layers: ', type(encoded_layers))
        # Each layer in the list is a torch tensor.
#         print('Tensor shape for each layer: ', encoded_layers[0].size())

    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(encoded_layers, dim=0)
#     print(token_embeddings.size())

    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
#     print(token_embeddings.size())

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
#     print(token_embeddings.size())
    
    return token_embeddings

In [15]:
def cat_last_four(token_embeddings): 
    # Concatenate the last 4 hidden layers to create contextual embeddings.
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat_last = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_last.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_last), len(token_vecs_cat_last[0])))
    print(token_vecs_cat_last[0])
    
    return token_vecs_cat_last

In [16]:
def cat_middle_four1(token_embeddings):
    # Concatenate the middle 4 hidden layers to create word embeddings.
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat_middle1 = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[4], token[5], token[6], token[7]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_middle1.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_middle1), len(token_vecs_cat_middle1[0])))
    print(token_vecs_cat_middle1[0])
    
    return token_vecs_cat_middle1

In [17]:
def cat_first_four(token_embeddings):
    token_vecs_cat_first = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[0], token[1], token[2], token[3]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_first.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_first), len(token_vecs_cat_first[0])))
    print(token_vecs_cat_first[0])
    
    return token_vecs_cat_first

In [18]:
def get_mean_embedding(token_embeddings):
    mean_embedding = sum(token_embeddings) / len(token_embeddings)
    print(mean_embedding)
    
    return mean_embedding

In [19]:
def reconstruct_tokens(tokenized_text):
    vocab_word = ''
    for i in tokenized_text:
        vocab_word += i.strip('#')
    print(vocab_word)

    return vocab_word

In [20]:
def write_embedding(embeddings_file, vocab_word, contextual_embedding):
    try:
        with open(embeddings_file, 'a') as f:
            f.write(vocab_word)
            for value in contextual_embedding[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {vocab_word}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

In [23]:
# if __name__ == "__main__":
# Set up input and output paths.
vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/vocab_files/vocab_checked.txt'
layer_combining_function = cat_middle_four1
embeddings_file = os.path.join('/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab', layer_combining_function.__name__ + '.txt')
if os.path.exists(embeddings_file):
    os.remove(embeddings_file)

    # Create a list of vocabulary words we want embeddings for.
vocab = make_vocab(vocab_file)

# Tokenize the vocabulary and look up the BERT token indices.
tokenized_text, indexed_tokens = tokenize_text(vocab)

# Generate segment IDs for each token.
segments_IDs = generate_segments_IDs(tokenized_text)

# Generate and write out the contextual embeddings for the vocabulary words.
# Embeddings are saved in a standard format that can be used for calcualting
# the cosine distances between word vectors.
for i in range(len(tokenized_text)):
    # Convert indexed tokens and segments to tensors.
    # Create a BERT model for the tokens.
    # Get the encoded model layers and reshape them.
    token_embeddings = generate_embeddings(indexed_tokens[i], segments_IDs[i])
    print(f'{tokenized_text[i]} has a token embedding of size {token_embeddings.size()}')

    # Extract the contextual embedding for a token.
    contextual_embedding = layer_combining_function(token_embeddings)

    # Write the embedding to a text file, with the vocabulary word prepended.
    vocab_word = reconstruct_tokens(tokenized_text[i])
    # Make sure we've got the correct vocabulary word.
    assert vocab[i] == vocab_word
    write_embedding(embeddings_file, vocab[i], contextual_embedding)

['aback'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.3298,  0.4090, -0.5590,  ..., -0.3376, -0.3176,  0.0872])
aback
Saved the embedding for aback.
['aba', '##shed'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.1815,  0.2656, -0.7644,  ..., -1.2575, -1.1458,  0.3598])
abashed
Saved the embedding for abashed.
['ab', '##hor'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.0624, -0.0087,  0.2358,  ..., -0.8970, -0.5811,  0.3655])
abhor
Saved the embedding for abhor.
['ab', '##hor', '##red'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.1223, -0.0474, -0.1086,  ..., -0.7768, -0.3913,  0.2025])
abhorred
Saved the embedding for abhorred.
['ab', '##hor', '##rence'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.2506,  0.1359, -0.1205,  ..., -1.0102, -0.0365,  0.6184])
abhorrence
Saved the embedding 

['ag', '##gra', '##vation'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.1651,  0.7620,  0.1863,  ...,  0.1485,  0.2556, -0.3368])
aggravation
Saved the embedding for aggravation.
['aggression'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.5449,  0.5502,  0.2965,  ..., -0.6756, -0.8149, -0.6476])
aggression
Saved the embedding for aggression.
['aggressive'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.2159,  0.4010,  0.4523,  ..., -0.0068, -0.6904, -0.1985])
aggressive
Saved the embedding for aggressive.
['ag', '##gr', '##ie', '##ve'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([ 0.2395,  0.8428,  0.0794,  ..., -0.5343,  0.2387,  0.1088])
aggrieve
Saved the embedding for aggrieve.
['ag', '##gr', '##ie', '##ved'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([ 0.1582,  0.8501,  0.0612,  ..., -1.126

['ang', '##st'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.0919, -0.0538,  0.7924,  ..., -0.5212, -0.3995,  0.1277])
angst
Saved the embedding for angst.
['anguish'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1329, -0.1217, -0.2044,  ..., -0.4078,  0.4993, -0.6908])
anguish
Saved the embedding for anguish.
['anguish', '##ed'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.1667, -0.0780,  0.1563,  ..., -0.0180,  0.2595, -0.5629])
anguished
Saved the embedding for anguished.
['animated'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1417, -0.8973, -0.4974,  ..., -0.0671, -0.4695,  0.0209])
animated
Saved the embedding for animated.
['an', '##imo', '##sity'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.1243,  0.7214,  0.0455,  ..., -0.5057, -0.8948,  1.1199])
animosity
Saved the embedding fo

['ashamed'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.2217, -0.0502,  0.2985,  ..., -0.5433, -0.9641, -0.2594])
ashamed
Saved the embedding for ashamed.
['aspiring'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1903,  0.1534,  0.5512,  ..., -0.1296, -1.0976, -0.0734])
aspiring
Saved the embedding for aspiring.
['assert', '##ive'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.0207,  0.3628,  0.3114,  ..., -0.4476, -1.1078, -0.6858])
assertive
Saved the embedding for assertive.
['assert', '##ively'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.1240,  0.4932,  0.0686,  ..., -0.3355, -0.7104, -1.1085])
assertively
Saved the embedding for assertively.
['assessing'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1840,  0.0447, -0.1364,  ..., -0.6361, -0.2406,  0.4616])
assessing
Saved the embedd

['beat'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.3468,  0.3886,  0.4718,  ...,  0.2080,  0.0875, -0.1007])
beat
Saved the embedding for beat.
['beaten'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.0346,  0.1107, -0.1936,  ...,  0.3533, -0.3622,  0.0366])
beaten
Saved the embedding for beaten.
['bed', '##ev', '##iled'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.2118,  0.1441,  0.0653,  ..., -0.0241, -0.5221, -0.1654])
bedeviled
Saved the embedding for bedeviled.
['be', '##fu', '##ddled'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.3803, -0.2611, -0.2467,  ..., -0.4744, -0.3737,  0.0625])
befuddled
Saved the embedding for befuddled.
['begging'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.3503,  0.8269,  0.5973,  ...,  0.5078, -0.0924, -0.7290])
begging
Saved the embedding for beggi

...

['un', '##cu', '##rio', '##us'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([-0.2441,  0.0896, -1.0311,  ..., -0.0097,  0.2784,  0.5465])
uncurious
Saved the embedding for uncurious.
['und', '##ec', '##ided'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.6635,  0.3374, -0.3854,  ...,  0.0876, -0.5049,  0.4243])
undecided
Saved the embedding for undecided.
['under', '##hand', '##ed'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.1627, -0.0654, -0.7017,  ...,  0.0025, -0.9553,  0.8798])
underhanded
Saved the embedding for underhanded.
['understanding'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.2486,  0.7745,  0.7579,  ...,  0.4240, -0.6672, -0.1672])
understanding
Saved the embedding for understanding.
['und', '##es', '##ira', '##ble'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([-0.3857,  0.3759

['un', '##res', '##olved'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.3734, -0.3464, -1.0613,  ...,  0.0439,  0.3188,  0.6544])
unresolved
Saved the embedding for unresolved.
['unrest', '##rained'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.0159,  0.5127,  0.8307,  ..., -0.8984, -0.7770, -0.5736])
unrestrained
Saved the embedding for unrestrained.
['un', '##ruff', '##led'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-7.9039e-01,  1.5122e-04, -1.4045e+00,  ...,  1.8214e-01,
         2.3196e-01,  3.0026e-01])
unruffled
Saved the embedding for unruffled.
['un', '##sat', '##is', '##fied'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([-9.7142e-04, -7.7382e-02, -1.1457e+00,  ..., -1.7301e-01,
        -2.4074e-01,  9.0319e-02])
unsatisfied
Saved the embedding for unsatisfied.
['un', '##sett', '##led'] has a token embedding of size torch.

['ve', '##xed'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.3974, -0.1700,  0.1356,  ...,  0.0364, -1.4226,  0.5406])
vexed
Saved the embedding for vexed.
['vicious'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.3645,  0.7932,  0.4546,  ..., -0.0928, -0.9104, -0.0656])
vicious
Saved the embedding for vicious.
['victorious'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.0088, -0.1459,  0.0980,  ..., -0.5282, -0.8872, -0.7480])
victorious
Saved the embedding for victorious.
['vi', '##gil', '##ant'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 4.0692e-02, -1.2690e-01,  2.3587e-01,  ..., -2.6046e-01,
        -3.5716e-01,  3.2761e-04])
vigilant
Saved the embedding for vigilant.
['vile'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.9707,  0.3933,  1.0464,  ..., -0.2730, -0.5877, -0.5175])
vile
Sav

['wicked'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.5564,  0.8588,  0.3279,  ...,  0.6275,  0.3978,  0.0556])
wicked
Saved the embedding for wicked.
['wild'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.0407,  0.2602, -0.0314,  ..., -0.6516, -0.6798,  0.1538])
wild
Saved the embedding for wild.
['will', '##ful'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.7357,  0.6843,  0.0975,  ...,  0.2560, -0.2128,  0.1917])
willful
Saved the embedding for willful.
['willing'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.0754,  0.1801,  0.8074,  ..., -0.5072, -0.7769, -0.0117])
willing
Saved the embedding for willing.
['wil', '##y'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.0458,  0.3267, -0.3480,  ..., -0.5323, -0.5748,  0.6810])
wily
Saved the embedding for wily.
['wink'] has a token embeddin

['yu', '##ck'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.5949,  0.2492,  0.6811,  ...,  0.0486, -0.6737,  0.6232])
yuck
Saved the embedding for yuck.
['za', '##ny'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.6550,  1.2369, -0.9386,  ..., -1.0238, -0.5476,  0.5598])
zany
Saved the embedding for zany.
['ze', '##alo', '##us'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.8600, -0.2303, -0.6630,  ..., -0.3600,  0.3167, -0.0689])
zealous
Saved the embedding for zealous.
['zen'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1668,  0.3055, -0.4071,  ..., -0.5739, -0.3290,  0.5896])
zen
Saved the embedding for zen.
['zone', '##d'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.4880,  0.6528,  0.2187,  ..., -0.4170, -0.9627, -0.3530])
zoned
Saved the embedding for zoned.
